#### Pablo Chantada Saborido (pablo.chantada@udc.es)

In [43]:
# Instalar dependecias si es necesario o hay algun fallo en las librerias

!pip install --quiet tensorflow tensorflow-datasets numpy matplotlib


In [44]:
# DEPENDENCIES

import tensorflow as tf
import tensorflow_datasets as tfds
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import pandas as pd

# Seteamos la semilla para repetición de los experimentos
# Al no tener que realizar el entrenamiento no sería necesario
seed=1234
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

### Carga de Datos

#### Normalización de Valores
Para evitar problemas relacionados con valores atípicos y garantizar un mejor procesamiento de las imágenes, normalizamos los valores. Dividimos todos los píxeles de las imágenes del dataset entre 127.5 _(la mitad de 255)_ para que los valores estén en el rango __[\-1, 1]__. Además, convertimos los valores a `float` para evitar cortes o redondeos que puedan afectar la calidad de las imágenes.

#### Importación del Dataset CIFAR-100
Cargamos el dataset CIFAR-100 y lo dividimos en conjuntos de entrenamiento (`train`) y prueba (`test`). Aplicamos la normalización y mejoramos el rendimiento mediante el uso del método `prefetch(tf.data.AUTOTUNE)`. Este método permite la paralelización, cargando el próximo lote mientras el modelo entrena con el lote actual.

In [45]:
def normalize_img(image, label):
    # Normalizamos las imágenes al rango [-1, 1]
    return tf.cast(image, tf.float32) / 127.5 - 1, label

def load_dataset(dataset="cifar10", batch_size = 128):

    try:
        ds_train, ds_test = tfds.load(
            dataset,
            split=['train', 'test'],
            as_supervised=True  # Devuelve el dataset como (imagen, label)
        )
    except Exception as e:
        raise ValueError(f"Error al cargar el dataset '{dataset}': {e}")

    # Preprocesado
    ds_train = ds_train.map(normalize_img).prefetch(tf.data.AUTOTUNE)
    ds_train = ds_train.shuffle(batch_size * 5).batch(batch_size).cache().repeat()
    ds_test = ds_test.map(normalize_img).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return ds_train, ds_test

ds_train, ds_test = load_dataset("cifar100")
# Comprobamos que las imagenes se cargen correctamente
for image, label in ds_train.take(1):
    print(f"Image shape: {image.shape}, Label shape: {label.shape}")

Image shape: (128, 32, 32, 3), Label shape: (128,)


2024-11-28 22:52:18.216710: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-11-28 22:52:18.216806: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [46]:
# Obtain a single batch from the dataset
for images, labels in ds_train.take(1):
    # Now you can access the shape of the images tensor
    print(images.shape[0])
    break

128


2024-11-28 22:52:18.266743: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-11-28 22:52:18.266834: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# Ejercicio 1 - Clase `ResidualBlock`

El funcionamiento del `ResidualBlock` es facilitar el aprendizaje en redes profundas combinando las características despues del procesado con la entrada original.

#### __Elementos__

1. __BatchNormalization:__
   - Normaliza los valores de la salida de las capas convolucionales.
   - Ayuda a reducir la sensibilidad del modelo a la inicialización de pesos y la tasa de aprendizaje.

2. __Función de Activación SiLU:__
   - Definida como $ \text{SiLU}(x) = x \cdot \sigma(x) $, donde $ \sigma(x) $ es la función sigmoide.

3. __Skip Connection:__
   - Permite que la entrada original del bloque se pase directamente a la salida, sumándose a las características procesadas.
   - Si las dimensiones de entrada y salida no coinciden, utiliza una convolución $ \times 1$ para ajustarlas.

4. __Convolución:__
   - Dos capas convolucionales $3 \times 3$ para procesar las características de la entrada.
   - La primera capa puede cambiar las dimensiones espaciales (si `strides` no es $(1, 1)$).
   - La segunda capa siempre mantiene las dimensiones de la salida.

5. __Suma final:__
   - Combina la salida de las capas convolucionales con la Skip Connection.

![Image](img/res_block.png)

In [47]:
class ResidualBlock(tf.keras.Model):
    def __init__(self, input_channels, output_channels, strides=(1, 1)):
        super(ResidualBlock, self).__init__()

        self.input_channels = input_channels
        self.output_channels = output_channels

        # Primera Convolución, no se utilizan bias
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.silu1 = tf.keras.layers.Activation('swish')  # SiLU = Swish activation
        self.conv1 = tf.keras.layers.Conv2D(
            filters=output_channels,
            kernel_size=(3, 3),
            strides=strides,
            padding='same',
            use_bias=False)

        # Segunda Convolución
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.silu2 = tf.keras.layers.Activation('swish')
        self.conv2 = tf.keras.layers.Conv2D(
            filters=output_channels,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            use_bias=False)

        # Skip Connection
        if input_channels != output_channels:
            self.skip_conv = tf.keras.layers.Conv2D(
                filters=output_channels,
                kernel_size=(1, 1),
                strides=strides,
                padding='same',
                use_bias=False)
        else:
            self.skip_conv = None

    def call(self, x):
        # Flujo principal
        x = self.bn1(x)
        x = self.silu1(x)
        residual = x

        # Comprobamos el valor de la Skip Connection
        if self.skip_conv:
            residual = self.skip_conv(residual)
        else:
            residual = residual

        # Continuación del flujo
        x = self.conv1(x)
        x = self.bn2(x)
        x = self.silu2(x)
        x = self.conv2(x)

        # Realizamos la suma de la Conv con la Skip Connection
        return tf.add(x, residual)


# Ejercicio 2 - Red `ResidualNetwork`

Creamos una red residual siguiendo el siguiente esquema:

![Image](img/network.png)

In [48]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='same', input_shape=(32, 32, 3), use_bias=False),
    ResidualBlock(16, 64),
    ResidualBlock(64, 64),
    ResidualBlock(64, 64),
    ResidualBlock(64, 128, strides=(2, 2)),

    ResidualBlock(128, 128),
    ResidualBlock(128, 128),
    ResidualBlock(128, 256, strides=(2, 2)),
    ResidualBlock(256, 256),
    ResidualBlock(256, 256),

    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(100, activation='softmax')
])


/home/clown/classroom/ap-2425-p1-ap-12-10/DL-gpu/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Al realizar el `Summary` del modelo podemos ver que es el mismo que en la imagen.

In [49]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_44 (Conv2D)              │ (None, 32, 32, 16)     │           432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_18               │ (None, 32, 32, 64)     │        47,424 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_19               │ (None, 32, 32, 64)     │        74,240 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_20               │ (None, 32, 32, 64)     │        74,240 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_21               │ (None, 16, 16, 128)    │       230,144 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_22               │ (None, 16, 16, 128)    │       295,936 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_23               │ (None, 16, 16, 128)    │       295,936 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_24               │ (None, 8, 8, 256)      │       919,040 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_25               │ (None, 8, 8, 256)      │     1,181,696 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_26               │ (None, 8, 8, 256)      │     1,181,696 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_56          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_56 (Activation)      │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        25,700 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,327,508 (16.51 MB)

 Trainable params: 4,322,100 (16.49 MB)

 Non-trainable params: 5,408 (21.12 KB)

# Carga del modelo

Cargamos los pesos dados por el profesor en nuestro modelo para comprobar su correcto funcionamiento. Para ello los compilaremos utilizando diferentes optimizadores y la métrica de `Accuracy` y `Loss` como referente. Para decir que obtenemos un buen funcionamiento, la accuracy en la evaluación ha de superar el $69%$.

### __Carga del Modelo y Evaluación__

Cargamos los pesos preentrenados proporcionados por el profesor en nuestro modelo. Esto permite verificar su correcto funcionamiento sin necesidad de entrenar desde cero. 

#### __Proceso:__
1. __Cargar Pesos:__ usamos la función dada por el profesor para cargar el modelo.
2. __Compilación:__ compilamos utilizando diferentes optimizadores y evaluando con las métricas `Accuracy` y `Loss`.
3. __Evaluación:__ para considerar que el modelo funciona correctamente, la `Accuracy` en el conjunto de evaluación debe superar el __69%__.

In [50]:
def load_weights(model, weight_file):
    '''
    Carga los pesos de un modelo utilizando pickle como Framework.
    '''
    with open(weight_file, 'rb') as f:
        weights = pickle.load(f)

    all_vars = model.trainable_weights + model.non_trainable_weights
    weight_list = [(x, weights[x]) for x in sorted(weights.keys())]
    weights = {}
    for i, var in enumerate(all_vars):
        aux = var.path.split('/')[-2:]
        classname = '_'.join(aux[0].split('_')[:-1])
        name = aux[1]
        assigned = False
        for j, (key, value) in enumerate(weight_list):
            if classname in key and name in key:
                try:
                    all_vars[i].assign(value)
                except:
                    continue
                # print('assigning', key, 'to', var.path)
                del weight_list[j]
                assigned = True
                break
        if not assigned:
            raise Exception(var.path + ' cannot be loaded')



In [51]:
load_weights(model, "weights.pkl")

# Esto es necesario ? xd

In [52]:
def one_hot_encode(image, label):
    label = tf.one_hot(label, depth=100)  # CIFAR-100 tiene 100 clases
    return image, label

In [53]:
def analize_model(model, optimizers, ds_test):
    """
    Evalúa el modelo con múltiples optimizadores en el conjunto de prueba.
    
    Args:
        model (tf.keras.Model): El modelo a evaluar.
        optimizers (dict): Diccionario con los optimizadores.
        ds_test (tf.data.Dataset): Dataset de prueba.

    Returns:
        pd.DataFrame: Resultados de la evaluación.
    """

    # Guardar pesos iniciales
    initial_weights = model.get_weights()

    metrics = [
        "accuracy",
        "sparse_categorical_accuracy"
    ]

    # Resultados
    results = []

    for opt_name, optimizer in optimizers.items():
        # Restaurar los pesos iniciales antes de la evaluación
        model.set_weights(initial_weights)

        # Compilar el modelo con la configuración actual
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=metrics
        )
        
        # Evaluar el modelo en el conjunto de prueba
        test_metrics = model.evaluate(ds_test, return_dict=True, verbose=0)
        
        # Guardar los resultados
        results.append({
            "optimizer": opt_name,
            **test_metrics
        })

    # Mostrar resultados en una tabla
    return pd.DataFrame(results)

# Analisis

1. __Accuracies y Losses:__
   - Las métricas de `accuracy` y `loss` son iguales independientemente del optimizador utilizado. 
   - Esto ocurre porque no realizamos el entrenamiento del modelo, sino que evaluamos directamente los pesos preentrenados.

2. __Funciones de Pérdida:__
   - Solo se utilizó la función de pérdida `sparse_categorical_crossentropy`, ya que las etiquetas están codificadas como valores enteros entre 0 y 99 para CIFAR-100.
   - No se probaron otras funciones de pérdida, como `categorical_crossentropy`, porque estas requieren etiquetas codificadas como *one-hot*, lo cual no se creyo relevante para este experimento.

3. __Conclusión:__
   - Los optimizadores no afectan los resultados cuando se evalúa un modelo preentrenado sin realizar entrenamiento.

# Ejercicio 3 - Última Capa


### Creación del `DataAumentation`

In [54]:
img_augmentation = tf.keras.models.Sequential(
    [
        tf.keras.layers.RandomRotation(2 * np.pi * 0.15), # rotaciones
        tf.keras.layers.RandomTranslation(0.1,0.1), # traslaciones
        tf.keras.layers.RandomFlip(), # giros
        tf.keras.layers.RandomContrast(0.1)  # contraste
    ],
    name="img_augmentation",
)

In [55]:
def freeze_model(model, show_layers=True):
    for layer in model.layers[:-1]:  # Congelar todas menos las últimas 'layers' capas
        if not isinstance(layer, tf.keras.layers.BatchNormalization):  # Excluye BatchNormalization
            layer.trainable = False
    for layer in model.layers[-1:]:  # Asegúrate de que las últimas capas son entrenables
        layer.trainable = True
    
    if show_layers is True:
        for layer in model.layers:
            print(f"{layer.name}: Trainable = {layer.trainable}")

In [56]:
# Definir las métricas
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

def train_step(x_batch, y_batch, model, loss_fn, optimizer):
    with tf.GradientTape() as tape:
        predictions = model(x_batch, training=True)
        loss = loss_fn(y_batch, predictions)

    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_loss.update_state(loss)
    train_accuracy.update_state(y_batch, predictions)

def train_model(model, train_dataset, loss_fn, optimizer, epochs=5, steps_per_epoch=390):
    train_dataset = train_dataset.repeat()  # Asegurar dataset infinito para steps_per_epoch
    history = {'accuracy': [], 'loss': []}

    for epoch in range(epochs):
        train_loss.reset_state()
        train_accuracy.reset_state()

        for step, (x_batch, y_batch) in enumerate(train_dataset):
            if step >= steps_per_epoch:
                break
            train_step(x_batch, y_batch, model, loss_fn, optimizer)

        # Resumen del epoch
        epoch_loss = train_loss.result().numpy()
        epoch_accuracy = train_accuracy.result().numpy()
        history['loss'].append(epoch_loss)
        history['accuracy'].append(epoch_accuracy)

        # Imprimir resumen de la época
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    return history


In [57]:
# Guardar los pesos iniciales
initial_weights = model.get_weights()
freeze_model(model)

conv2d_44: Trainable = False
residual_block_18: Trainable = False
residual_block_19: Trainable = False
residual_block_20: Trainable = False
residual_block_21: Trainable = False
residual_block_22: Trainable = False
residual_block_23: Trainable = False
residual_block_24: Trainable = False
residual_block_25: Trainable = False
residual_block_26: Trainable = False
batch_normalization_56: Trainable = True
activation_56: Trainable = False
global_average_pooling2d_2: Trainable = False
dense_2: Trainable = True


In [63]:
ds_train, ds_test = load_dataset("cifar10")
# ds_train = ds_train.map(lambda x, y: (img_augmentation(x, training=True), y))


def train_with_optimizers(model, ds_train, optimizers, epochs=5, batch_size=128, train_steps_per_epoch=None, mode="fit"):
    """
    Entrena el modelo con múltiples optimizadores y analiza los resultados.
    
    Args:
        model (tf.keras.Model): El modelo a entrenar.
        ds_train (tf.data.Dataset): Dataset de entrenamiento.
        optimizers (dict): Diccionario con los optimizadores.
        epochs (int): Número de épocas de entrenamiento.
        batch_size (int): Tamaño de los lotes.
        train_steps_per_epoch (int): Número de pasos por época (opcional).
        mode (str): Modo de entrenamiento, "fit" o "manual".

    Returns:
        pd.DataFrame: Resultados del entrenamiento con diferentes optimizadores.
    """

    # Guardar pesos iniciales
    initial_weights = model.get_weights()

    # Resultados
    results = []

    # Determinar pasos por época si no se especifican
    if train_steps_per_epoch is None:
        train_steps_per_epoch = 50000 // batch_size

    for opt_name, optimizer in optimizers.items():
        print(f"\nEntrenando con optimizador: {opt_name}")

        # Restaurar los pesos iniciales antes de cada entrenamiento
        model.set_weights(initial_weights)

        # Configurar el modelo
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"]
        )
        
        history = model.fit(
            ds_train,
            epochs=epochs,
            steps_per_epoch=train_steps_per_epoch,
            verbose=0
        )
        # Extraer métricas de la última época
        final_train_acc = history.history["accuracy"][-1]
        final_train_loss = history.history["loss"][-1]
        # Registrar resultados
        results.append({
            "optimizer": opt_name,
            "final_train_accuracy": final_train_acc,
            "final_train_loss": final_train_loss,
        })
        
        results_df1 = analize_model(model, optimizers, ds_test)
        print("Resultados de Test {opt_name}:")
        print(results_df1.to_string(index=False, justify="left"))


    # Mostrar resultados en un DataFrame
    return pd.DataFrame(results)

In [64]:
optimizers = {
    "Adam": tf.keras.optimizers.Adam(learning_rate=1e-5),
    "SGD": tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9),
    "RMSprop": tf.keras.optimizers.RMSprop(learning_rate=5e-6)
}


results_df = train_with_optimizers(model, ds_train, optimizers, epochs=5, batch_size=128, mode="fit")
print("Resultados de Modelos:")
print(results_df.to_string(index=False, justify="left"))



Entrenando con optimizador: Adam
Resultados de Test {opt_name}:
optimizer  accuracy  loss     sparse_categorical_accuracy
   Adam   0.0806    4.966501 0.0806                      
    SGD   0.0806    4.966501 0.0806                      
RMSprop   0.0806    4.966501 0.0806                      

Entrenando con optimizador: SGD
Resultados de Test {opt_name}:
optimizer  accuracy  loss     sparse_categorical_accuracy
   Adam   0.6867    0.875939 0.6867                      
    SGD   0.6867    0.875939 0.6867                      
RMSprop   0.6867    0.875939 0.6867                      

Entrenando con optimizador: RMSprop
Resultados de Test {opt_name}:
optimizer  accuracy  loss     sparse_categorical_accuracy
   Adam   0.0433    6.215019 0.0433                      
    SGD   0.0433    6.215019 0.0433                      
RMSprop   0.0433    6.215019 0.0433                      
Resultados de Modelos:
optimizer  final_train_accuracy  final_train_loss
   Adam   0.077759              5.

In [ ]:
# Restaurar pesos iniciales
model.set_weights(initial_weights)

# Crear instancia del optimizador Adam
optimizer_adam = tf.keras.optimizers.Adam(learning_rate=1e-5)
batch_size = 128

# Entrenar el modelo
history_adam = train_model(
    model=model,
    train_dataset=ds_train,
    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=optimizer_adam,
    epochs=5,
    steps_per_epoch=50000//batch_size
)

# Mostrar resultados de Adam
print("Resultados - Optimizador Adam:")
print(f"Accuracy final: {history_adam['accuracy'][-1]:.4f}")
print(f"Loss final: {history_adam['loss'][-1]:.4f}")


KeyboardInterrupt: 

In [62]:
# Restaurar pesos iniciales
model.set_weights(initial_weights)

# Crear instancia del optimizador SGD
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9)

batch_size = 128

# Entrenar el modelo
history_sgd = train_model(
    model=model,
    train_dataset=ds_train,
    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=optimizer_adam,
    epochs=5,
    steps_per_epoch=50000//batch_size
)
# Mostrar resultados de SGD
print("Resultados - Optimizador SGD:")
print(f"Accuracy final: {history_sgd['accuracy'][-1]:.4f}")
print(f"Loss final: {history_sgd['loss'][-1]:.4f}")


ValueError: Unknown variable: <KerasVariable shape=(256,), dtype=float32, path=sequential_2/batch_normalization_56/gamma>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should recreate a new optimizer instance.

In [ ]:
# Restaurar pesos iniciales
model.set_weights(initial_weights)

# Crear instancia del optimizador RMSprop
optimizer_rmsprop = tf.keras.optimizers.RMSprop(learning_rate=1e-6)

# Entrenar el modelo
history_rmsprop = train_model(
    model=model,
    train_dataset=ds_train,
    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=optimizer_adam,
    epochs=5,
    steps_per_epoch=50000//batch_size
)
# Mostrar resultados de RMSprop
print("Resultados - Optimizador RMSprop:")
print(f"Accuracy final: {history_rmsprop['accuracy'][-1]:.4f}")
print(f"Loss final: {history_rmsprop['loss'][-1]:.4f}")


Epoch 1/5 - Loss: 4.0423, Accuracy: 0.1105
Epoch 2/5 - Loss: 3.1234, Accuracy: 0.1451
Epoch 3/5 - Loss: 2.7061, Accuracy: 0.1798


2024-11-28 21:29:59.271758: I tensorflow/core/framework/local_rendezvous.cc:428] Local rendezvous send item cancelled. Key hash: 7920739947562291792
2024-11-28 21:29:59.271811: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12016093490960807282
2024-11-28 21:29:59.271826: I tensorflow/core/framework/local_rendezvous.cc:428] Local rendezvous send item cancelled. Key hash: 14345683130608024958
2024-11-28 21:29:59.271844: I tensorflow/core/framework/local_rendezvous.cc:428] Local rendezvous send item cancelled. Key hash: 13898018971219156263
2024-11-28 21:29:59.271900: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12246332304163951425
2024-11-28 21:29:59.271921: I tensorflow/core/framework/local_rendezvous.cc:428] Local rendezvous send item cancelled. Key hash: 16739058962029659819
2024-11-28 21:29:59.271931: I tensorflow/core/framework/local_rendezvous.cc:428] Local rendezvous sen

Epoch 4/5 - Loss: 2.4572, Accuracy: 0.2047
Epoch 5/5 - Loss: 2.2902, Accuracy: 0.2300
Resultados - Optimizador RMSprop:
Accuracy final: 0.2300
Loss final: 2.2902


# Ejercicio 4 - Toda la Red

In [65]:
def freeze_model(model, show_layers=True):

    for layer in model.layers[:-1]:  # Todas menos la última capa
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True 
        else:
            layer.trainable = False  # Asegurar que BatchNormalization también esté congelada

    # Hacer la última capa entrenable
    model.layers[-1].trainable = False
    
    if show_layers is True:
        for layer in model.layers:
            print(f"{layer.name}: Trainable = {layer.trainable}")


In [66]:
model.set_weights(initial_weights)
freeze_model(model)

conv2d_44: Trainable = True
residual_block_18: Trainable = True
residual_block_19: Trainable = True
residual_block_20: Trainable = True
residual_block_21: Trainable = True
residual_block_22: Trainable = True
residual_block_23: Trainable = True
residual_block_24: Trainable = True
residual_block_25: Trainable = True
residual_block_26: Trainable = True
batch_normalization_56: Trainable = False
activation_56: Trainable = True
global_average_pooling2d_2: Trainable = True
dense_2: Trainable = False


In [67]:
optimizers = {
    "Adam": tf.keras.optimizers.Adam(learning_rate=1e-7),
    "SGD": tf.keras.optimizers.SGD(learning_rate=1e-5, momentum=0.9),
    "RMSprop": tf.keras.optimizers.RMSprop(learning_rate=5e-8)
}


results_df = train_with_optimizers(model, ds_train, optimizers, epochs=5, batch_size=128, mode="fit")
print("Resultados de Modelos:")
print(results_df.to_string(index=False, justify="left"))


Entrenando con optimizador: Adam
Resultados de Test {opt_name}:
optimizer  accuracy  loss     sparse_categorical_accuracy
   Adam   0.0605    5.706727 0.0605                      
    SGD   0.0605    5.706727 0.0605                      
RMSprop   0.0605    5.706727 0.0605                      

Entrenando con optimizador: SGD
Resultados de Test {opt_name}:
optimizer  accuracy  loss     sparse_categorical_accuracy
   Adam   0.6482    1.306507 0.6482                      
    SGD   0.6482    1.306507 0.6482                      
RMSprop   0.6482    1.306507 0.6482                      

Entrenando con optimizador: RMSprop
Resultados de Test {opt_name}:
optimizer  accuracy  loss     sparse_categorical_accuracy
   Adam   0.0199    7.067215 0.0199                      
    SGD   0.0199    7.067215 0.0199                      
RMSprop   0.0199    7.067215 0.0199                      
Resultados de Modelos:
optimizer  final_train_accuracy  final_train_loss
   Adam   0.052174              5.

In [ ]:
# Ejemplo de uso
model.set_weights(initial_weights)

results_df = train_with_optimizers(model, ds_train, ds_test, epochs=25, batch_size=128, mode="fit")
print("Resultados de Modelos:")
print(results_df.to_string(index=False, justify="left"))

results_df1 = analize_model(model)
print("Resultados de Test Modelos:")
print(results_df1.to_string(index=False, justify="left"))




Entrenando con optimizador: Adam
Epoch 1/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 32s 65ms/step - accuracy: 0.5556 - loss: 1.2498
Epoch 2/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 28s 64ms/step - accuracy: 0.5809 - loss: 1.1817
Epoch 3/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 25s 64ms/step - accuracy: 0.5959 - loss: 1.1394
Epoch 4/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 25s 64ms/step - accuracy: 0.6118 - loss: 1.0997
Epoch 5/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 25s 64ms/step - accuracy: 0.6213 - loss: 1.0693

Entrenando con optimizador: SGD
Epoch 1/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 30s 62ms/step - accuracy: 0.3417 - loss: 2.1184
Epoch 2/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 26s 62ms/step - accuracy: 0.4920 - loss: 1.4212
Epoch 3/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 24s 62ms/step - accuracy: 0.5363 - loss: 1.3121
Epoch 4/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 24s 62ms/step - accuracy: 0.5568 - loss: 1.2430
Epoch 5/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 24s 62ms/step - accuracy: 0.5790 - loss: 1.1888

Entrenando con optimizador: RMSprop
Epoch 1/5
390/390 ━━━━━━━━

In [ ]:
# Restaurar pesos iniciales
model.set_weights(initial_weights)

# Crear instancia del optimizador Adam
optimizer_adam = tf.keras.optimizers.Adam(learning_rate=1e-7)
batch_size = 128

# Entrenar el modelo
history_adam2 = train_model(
    model=model,
    train_dataset=ds_train,
    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=optimizer_adam,
    epochs=5,
    steps_per_epoch=50000//batch_size
)

# Mostrar resultados de Adam
print("Resultados - Optimizador Adam:")
print(f"Accuracy final: {history_adam['accuracy'][-1]:.4f}")
print(f"Loss final: {history_adam['loss'][-1]:.4f}")


Epoch 1/5 - Loss: 7.1867, Accuracy: 0.0143
Epoch 2/5 - Loss: 6.8523, Accuracy: 0.0192
Epoch 3/5 - Loss: 6.5359, Accuracy: 0.0240
Epoch 4/5 - Loss: 6.2132, Accuracy: 0.0322
Epoch 5/5 - Loss: 5.9491, Accuracy: 0.0407
Resultados - Optimizador Adam:
Accuracy final: 0.2067
Loss final: 2.4496


In [ ]:
# Restaurar pesos iniciales
model.set_weights(initial_weights)

# Crear instancia del optimizador SGD
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate=1e-5, momentum=0.9)

batch_size = 128

# Entrenar el modelo
history_sgd = train_model(
    model=model,
    train_dataset=ds_train,
    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=optimizer_adam,
    epochs=5,
    steps_per_epoch=50000//batch_size
)
# Mostrar resultados de SGD
print("Resultados - Optimizador SGD:")
print(f"Accuracy final: {history_sgd['accuracy'][-1]:.4f}")
print(f"Loss final: {history_sgd['loss'][-1]:.4f}")


Epoch 1/5 - Loss: 7.1747, Accuracy: 0.0145
Epoch 2/5 - Loss: 6.8194, Accuracy: 0.0189
Epoch 3/5 - Loss: 6.4920, Accuracy: 0.0266
Epoch 4/5 - Loss: 6.2007, Accuracy: 0.0320
Epoch 5/5 - Loss: 5.9170, Accuracy: 0.0421
Resultados - Optimizador SGD:
Accuracy final: 0.0421
Loss final: 5.9170


In [ ]:
# Restaurar pesos iniciales
model.set_weights(initial_weights)

# Crear instancia del optimizador RMSprop
optimizer_rmsprop = tf.keras.optimizers.RMSprop(learning_rate=1e-7)

# Entrenar el modelo
history_rmsprop = train_model(
    model=model,
    train_dataset=ds_train,
    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=optimizer_adam,
    epochs=5,
    steps_per_epoch=50000//batch_size
)
# Mostrar resultados de RMSprop
print("Resultados - Optimizador RMSprop:")
print(f"Accuracy final: {history_rmsprop['accuracy'][-1]:.4f}")
print(f"Loss final: {history_rmsprop['loss'][-1]:.4f}")


Epoch 1/5 - Loss: 7.1685, Accuracy: 0.0151


KeyboardInterrupt: 